In [ ]:
import gzip
import glob
import json
import pandas as pd
import numpy as np

In [ ]:
path2 = 'labels_predictions_twitter_test_images'

In [ ]:
df2 = pd.read_csv(path2)

In [ ]:
df2.head()

In [ ]:
data_df = pd.DataFrame()

In [ ]:
data_df['label'] = df2['label']
data_df['prediction'] = df2['prediction']

In [ ]:
data_df.head()

In [ ]:
probas = np.array(data_df['prediction'].to_list())
labels = (probas > 0.5).astype(np.int)
print(labels)

In [ ]:
sum(labels)

In [ ]:
race_dict = {}
races = list(labels)
user_ids = data_df['label'].to_list()
for i in range(len(user_ids)):
    race_dict[int(user_ids[i])] = races[i]

twitter_race_tweets = {}

In [ ]:
path = 'Twitter_Covid_Data/*.json.gz'
files = glob.glob(path)

In [ ]:
df_race_tweets = pd.DataFrame()

In [ ]:
for i in range(len(files)):
    try:
        with gzip.open(files[i], 'rb') as f:
            for line in f:
                data = json.loads(line)
                try:
                    uid = data['user']['id']
                except:
                    continue
                if uid in race_dict.keys():
                    twitter_race_tweets['user_id'] = uid
                    twitter_race_tweets['race'] = race_dict[uid]
                    if data['truncated'] == True:
                        twitter_race_tweets['tweet'] = data['extended_tweet']['full_text']
                    else:
                        twitter_race_tweets['tweet'] = data['text']
                    df_race_tweets = df_race_tweets.append(twitter_race_tweets, ignore_index=True)
                    df_race_tweets['race'] = df_race_tweets['race'].astype('int')
                    df_race_tweets['user_id'] = df_race_tweets['user_id'].astype('int')
                    twitter_race_tweets.clear()
    except Exception as e:
        print(e)
        print("Faulty file ", files[i]) #some files are faulty in the retrieved dataset and do not open



In [ ]:
df_race_tweets

In [ ]:
df_race_tweets.to_csv('output_twitter_race_tweets.csv')